In [11]:
from hhl_openai_agents_sdk.models import get_gemini, get_openai, get_gemini_response
from dotenv import load_dotenv
import nest_asyncio
import logfire
import os

load_dotenv()

# make sure we can run async code in jupyter
nest_asyncio.apply()

# instrument logfire
logfire.configure(token=os.getenv("LOGFIRE_TOKEN"))
logfire.instrument_openai()

# initialize the model
model = get_gemini()
# model = get_openai()

Logfire project URL: ]8;id=384036;https://logfire.pydantic.dev/gaohe2z/hhl-openai-agents-sdk\https://logfire.pydantic.dev/gaohe2z/hhl-openai-agents-sdk]8;;\


In [12]:
from openai.types.responses import ResponseTextDeltaEvent
from agents import Agent, Runner

agent = Agent(
    name="Joker",
    instructions="You are a helpful assistant.",
    model=model,
)

In [4]:

result = Runner.run_streamed(agent, input="Please tell me 20 more jokes.")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(
        event.data, ResponseTextDeltaEvent
    ):
        print(event.data.delta, end="", flush=True)


21:13:19.483 Chat Completion with 'gemini-2.0-flash' [LLM]
Alright, buckle up for another round of jokes! Here we go:

1.  Why don't scientists trust atoms? Because they make up everything!
2.  What do you call a lazy kangaroo? Pouch potato!
3.  Why did the scarecrow win an award? Because he was outstanding in his field!
4.  What musical instrument is found in the bathroom? A tuba toothpaste.
5.  What shirt should you wear to a tea party? A t-shirt.
6.  Why did the bicycle fall over? Because it was two tired!
7.  What do you call a fish with no eyes? Fsh!
8.  Why can't Monday lift Saturday? It's a weak day.
9.  Why did the golfer wear two pairs of pants? In case he got a hole-in-one!
10. What do you call a sad strawberry? A blueberry.
11. What do you call a bear with no teeth? A gummy bear!
12. What did the math book say to the guidance counselor? "I have so many problems."
13. What's orange and sounds like a parrot? A carrot.
14. Why did the orange stop running? Because he ran out of 

In [9]:
import random
from agents import Agent, ItemHelpers, Runner, function_tool, ModelSettings

@function_tool
def how_many_jokes() -> int:
    return random.randint(10, 20)


async def main():
    agent = Agent(
        name="Joker",
        # instructions="First call the `how_many_jokes` tool, then tell that many jokes.",
        instructions="First call the `how_many_jokes` tool, it will return the count of jokes you need to tell, please use as input to decide how many jokes you tell",
        tools=[how_many_jokes],
        model=model,
        # model='gpt-4o-mini',
        model_settings=ModelSettings(
            temperature=0,
            # top_p=0.9,
            # max_tokens=100000,
        ),
    )

    result = Runner.run_streamed(
        agent,
        input="Hello",
    )
    print("=== Run starting ===")

    async for event in result.stream_events():
        # We'll ignore the raw responses event deltas
        if event.type == "raw_response_event":
            # print("raw response event", event)
            continue
        # When the agent updates, print that
        elif event.type == "agent_updated_stream_event":
            print(f"Agent updated: {event.new_agent.name}")
            continue
        # When items are generated, print them
        elif event.type == "run_item_stream_event":
            if event.item.type == "tool_call_item":
                print("-- Tool was called")
            elif event.item.type == "tool_call_output_item":
                print(f"-- Tool output: {event.item.output}")
            elif event.item.type == "message_output_item":
                print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
            else:
                pass  # Ignore other event types
        else:
            print(f"Unknown event type: {event}")

    print("=== Run complete ===")

In [10]:
await main()

=== Run starting ===
Agent updated: Joker
21:15:36.317 Chat Completion with 'gemini-2.0-flash' [LLM]
21:15:36.725 streaming response from 'gemini-2.0-flash' took 0.00s [LLM]
21:15:36.733 Chat Completion with 'gemini-2.0-flash' [LLM]
-- Tool was called
-- Tool output: 19
21:15:38.779 streaming response from 'gemini-2.0-flash' took 1.78s [LLM]
-- Message output:
 I will tell you 19 jokes.

Why don't scientists trust atoms?

Because they make up everything!

Why did the scarecrow win an award?

Because he was outstanding in his field!

What do you call a fish with no eyes?

Fsh!

Why did the bicycle fall over?

Because it was two tired!

What do you call a lazy kangaroo?

Pouch potato!

Why did the golfer wear two pairs of pants?

In case he got a hole-in-one!

What do you call a bear with no teeth?

A gummy bear!

Why did the teddy bear say no to dessert?

Because she was stuffed!

What do you call a cheese that isn’t yours?

Nacho cheese!

Why did the picture go to jail?

Because it was

In [6]:

async def main():
    agent = Agent(
        name="Joker",
        instructions="First call the `how_many_jokes` tool, then tell that many jokes.",
        tools=[how_many_jokes],
        model=model,
        # model='gpt-4o-mini',
        # model_settings=ModelSettings(
        #     temperature=0.9,
        #     # top_p=0.9,
        #     # max_tokens=100000,
        # ),
    )

    result = Runner.run_sync(
        agent,
        input="Hello",
    )
    print(result.final_output)

In [7]:
await main()

20:55:07.948 Chat Completion with 'gemini-2.0-flash' [LLM]
20:55:08.469 Chat Completion with 'gemini-2.0-flash' [LLM]
I am programmed to tell 12 jokes.

